In [7]:
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
import yfinance
import json


def get_ticker(inputs):
    ddg = DuckDuckGoSearchAPIWrapper()
    company_name = inputs["company_name"]
    return ddg.run(f"Ticker symbol of {company_name}")


def get_income_statement(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.income_stmt.to_json())


def get_balance_sheet(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.balance_sheet.to_json())


def get_daily_stock_performance(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.history(period="3mo").to_json())


functions = [
    {
        "type": "function",
        "function": {
            "name": "get_ticker",
            "description": "Given the name of a company, this function returns its ticker symbol",
            "parameters": {
                "type": "object",
                "properties": {
                    "company_name": {
                        "type": "string",
                        "description": "The name of the company",
                    },
                },
                "required": ["company_name"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_income_statement",
            "description": "Given a ticker symbol (e.g., APPL), this function returns the copmany's income statement.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_balance_sheet",
            "description": "Given a ticker symbol (e.g., AAPL), the funciton returns the company's balance sheet.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_daily_stock_performance",
            "description": "Given a ticker symbol (e.g., AAPL), the function returns the performance of the stock for the last 100 days.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the companny",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
]

In [2]:
from openai import OpenAI

client = OpenAI()

assistant = client.beta.assistants.create(
    name="Investor Assistant",
    instructions="You help users perform resarch on publically traded companies to decide if they should buy the stock or not.",
    tools=functions,  # Look above
    model="gpt-4o",
)

assistant_id = "asst_zXuOTWn59mey5N5rmEh2G8Ex"

In [3]:
# client = OpenAI()

thread = client.beta.threads.create()

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I want to know if Health Catalyst stock is a good buy.",
)

thread

Thread(id='thread_aMhQjjWmRy1OGzcaOAo4zt3U', created_at=1720306617, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))

In [18]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant_id,
)

run

Run(id='run_XmM7vWMwvnG9RK7GbajomOGe', assistant_id='asst_zXuOTWn59mey5N5rmEh2G8Ex', cancelled_at=None, completed_at=None, created_at=1720307266, expires_at=1720307866, failed_at=None, incomplete_details=None, instructions='You help users perform resarch on publically traded companies to decide if they should buy the stock or not.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_aMhQjjWmRy1OGzcaOAo4zt3U', tool_choice='auto', tools=[FunctionTool(function=FunctionDefinition(name='get_ticker', description='Given the name of a company, this function returns its ticker symbol', parameters={'type': 'object', 'properties': {'company_name': {'type': 'string', 'description': 'The name of the company'}}, 'required': ['copmany_name']}), type='function'), FunctionTool(function=FunctionDefinition(name='get

In [30]:
def get_run(run_id, thread_id):
    return client.beta.threads.runs.retrieve(run_id=run_id, thread_id=thread_id)


def get_messsages(thread_id):
    messages = client.beta.threads.messages.list(thread_id=thread_id)
    messages = list(messages)
    messages.reverse()
    for message in messages:
        print(f"{message.role}: {message.content[0].text.value}")

In [25]:
import json
import time


def wait_for_run_completion(thread_id, run_id, timeout=300):
    start_time = time.time()
    while time.time() - start_time < timeout:
        run = get_run(run_id, thread_id)
        if run.status == "completed":
            return run
        elif run.status == "requires_action":
            handle_required_actions(thread_id, run)
        elif run.status in ["failed", "cancelled", "expired"]:
            raise Exception(f"Run ended with status: {run.status}")
        time.sleep(1)  # Wait for a second before checking again
    raise TimeoutError("Run did not complete within the specified timeout.")


def handle_required_actions(thread_id, run):
    tool_calls = run.required_action.submit_tool_outputs.tool_calls
    tool_outputs = []

    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_args = json.loads(tool_call.function.arguments)

        # Execute the appropriate function based on the function name
        if function_name == "get_ticker":
            output = get_ticker(function_args)
        elif function_name == "get_income_statement":
            output = get_income_statement(function_args)
        elif function_name == "get_balance_sheet":
            output = get_balance_sheet(function_args)
        elif function_name == "get_daily_stock_performance":
            output = get_daily_stock_performance(function_args)
        else:
            output = f"Function {function_name} not found"

        tool_outputs.append({"tool_call_id": tool_call.id, "output": output})

    # Submit the outputs back to the assistant
    client.beta.threads.runs.submit_tool_outputs(
        thread_id=thread_id, run_id=run.id, tool_outputs=tool_outputs
    )


# Use this function to wait for the run to complete
try:
    completed_run = wait_for_run_completion(thread.id, run.id)
    print("Run completed successfully")

    # Retrieve and print the messages
    messages = client.beta.threads.messages.list(thread_id=thread.id)
    for message in messages:
        print(f"{message.role}: {message.content[0].text.value}")
except Exception as e:
    print(f"An error occurred: {str(e)}")

Run completed successfully
assistant: To determine if Health Catalyst (HCAT) stock is a good buy, let's analyze its recent financial performance, balance sheet, and stock performance:

### Financial Performance (Income Statement)

#### Revenue and Profitability:
- **2023:**
  - Total Revenue: $295.938 million
  - Gross Profit: $131.833 million
  - Net Income: -$118.147 million
  - EBITDA: -$68.281 million
- **2022:**
  - Total Revenue: $276.236 million
  - Gross Profit: $133.187 million
  - Net Income: -$137.403 million
  - EBITDA: -$86.147 million
- **2021:**
  - Total Revenue: $241.926 million
  - Gross Profit: $117.572 million
  - Net Income: -$153.210 million
  - EBITDA: -$106.122 million

#### Observations:
- The company has been consistently increasing its revenue over the past three years.
- Despite an increase in revenue, Health Catalyst has been operating at a net loss each year, which is increasing slightly year over year.

### Balance Sheet

#### Key Metrics:
- **2023:**
  -

In [31]:
get_messsages(thread.id)

user: I want to know if Health Catalyst stock is a good buy.
assistant: To determine if Health Catalyst (HCAT) stock is a good buy, let's analyze its recent financial performance, balance sheet, and stock performance:

### Financial Performance (Income Statement)

#### Revenue and Profitability:
- **2023:**
  - Total Revenue: $295.938 million
  - Gross Profit: $131.833 million
  - Net Income: -$118.147 million
  - EBITDA: -$68.281 million
- **2022:**
  - Total Revenue: $276.236 million
  - Gross Profit: $133.187 million
  - Net Income: -$137.403 million
  - EBITDA: -$86.147 million
- **2021:**
  - Total Revenue: $241.926 million
  - Gross Profit: $117.572 million
  - Net Income: -$153.210 million
  - EBITDA: -$106.122 million

#### Observations:
- The company has been consistently increasing its revenue over the past three years.
- Despite an increase in revenue, Health Catalyst has been operating at a net loss each year, which is increasing slightly year over year.

### Balance Sheet


In [26]:
get_run(run.id, thread.id).status

'completed'